<a href="https://colab.research.google.com/github/norman-AI-2025/hackathon-2025/blob/main/Loan_Risk_App_merged_V11_3_(final_product).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess
import time
import os
import sys
import re
import urllib.request

# ==========================================
# PART 1: CLEANUP & SETUP
# ==========================================

print("🧹 Cleaning up old processes...")
subprocess.run(["pkill", "cloudflared"])
subprocess.run(["pkill", "streamlit"])
time.sleep(2)

print("📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "streamlit", "pandas", "transformers", "torch", "plotly"])

# ==========================================
# PART 2: CREATE APP FILE
# ==========================================

print("📝 Writing application file...")

loan_risk_app_code = '''
import pandas as pd
import streamlit as st
import numpy as np
import time
import plotly.graph_objects as go
from datetime import date

# --- CONSTANTS ---
PAGES_PERSONAL = {1: "Applicant Info", 2: "Guarantor Info", 3: "Loan Details", 4: "Results"}
GENDER_OPTIONS = ["Male", "Female"]
MARITAL_OPTIONS = ["Single", "Married", "Divorced", "Widowed"]
EDUCATION_OPTIONS = ["Secondary School", "Diploma", "Bachelor's Degree", "Master's/PhD", "Professional Qual."]
EMPLOYMENT_OPTIONS = ["Salaried (MNC/Gov)", "Salaried (SME)", "Self-Employed", "Business Owner", "Unemployed", "Retired"]
HOUSING_OPTIONS = ["Owned (Mortgaged)", "Owned (Free from Mortgage)", "Rented", "Living with Parents/Relatives", "Employer Quarters"]
RELATIONSHIP_OPTIONS = ["Spouse", "Parent", "Sibling", "Business Partner", "Friend"]
COLLATERAL_OPTIONS = ["Residential Property", "Commercial Property", "Vehicle", "Fixed Deposit/Cash", "None"]

INDUSTRY_RISK = {
    "Government/GLC": 1,
    "Healthcare/Medical": 2,
    "Financial Services": 3,
    "Education": 3,
    "Manufacturing": 5,
    "Retail/F&B": 7,
    "Construction/Real Estate": 8,
    "Tourism/Hospitality": 9,
    "Oil & Gas": 6
}

# --- KEYWORD DATA (INTEGRATED) ---
RAW_RISK_DATA = """
fraud|T3
fraudulent|T3
scam|T3
scammer|T3
scamming|T3
ponzi|T3
pyramidscheme|T3
ponzischeme|T3
launder|T3
laundering|T3
moneylaundering|T3
forgery|T3
forged|T3
counterfeit|T3
fakeid|T3
fakeic|T3
stolenid|T3
stolenic|T3
bribe|T3
bribery|T3
kickback|T3
taxevasion|T3
evadetax|T3
ghostaccount|T3
shellcompany|T3
dummycompany|T3
fake|T2
bogus|T2
shady|T2
illicit|T2
illegal|T2
underhand|T2
deception|T2
deceive|T2
cheat|T2
cheating|T2
dishonest|T2
falsify|T2
falsified|T2
misrepresent|T2
misrepresentation|T2
suspicious|T1
dodgy|T1
manipulated|T1
manipulation|T1
rigged|T1
drugdealer|T3
drugdealing|T3
drugtrade|T3
drugs|T3
narcotics|T3
trafficking|T3
smuggling|T3
smuggler|T3
contraband|T3
cartel|T3
mafia|T3
brothel|T3
prostitution|T3
pimp|T3
kidnapping|T3
kidnap|T3
ransom|T3
terrorist|T3
terrorism|T3
extremist|T3
extremism|T3
isis|T3
alqaeda|T3
bomb|T3
bombing|T3
grenade|T3
explosives|T3
weaponstrade|T3
armsdeal|T3
weaponsdeal|T3
gangster|T2
gang|T2
triad|T2
syndicate|T2
underground|T2
blackmarket|T2
outlaw|T2
fugitive|T2
weapon|T2
weapons|T2
firearm|T2
ammo|T2
gun|T2
radical|T2
radicalized|T2
bomber|T2
crime|T1
criminal|T1
violence|T1
gambling|T2
gamble|T2
casino|T2
betting|T2
bettor|T2
jackpot|T2
slots|T2
slot|T2
bookie|T2
roulette|T2
poker|T2
lottery|T2
addiction|T2
addict|T2
desperate|T2
desperation|T2
highdebt|T2
heavilyindebted|T2
overleveraged|T2
overleverage|T2
overdue|T2
arrears|T2
delinquent|T2
default|T2
defaulted|T2
bankrupt|T2
bankruptcy|T2
insolvent|T2
insolvency|T2
broke|T1
stressdebt|T1
debtproblem|T1
abscond|T3
absconded|T3
absconding|T3
runaway|T3
跑路|T3
逃跑|T3
vanish|T2
disappear|T2
ponzischeme|T3
pyramidscheme|T3
hyip|T3
highyield|T3
loanshark|T3
loan-shark|T3
ahlong|T3
ah-long|T3
ceti|T3
samseng|T3
lintahdarah|T3
along|T3
mlm|T2
binary|T2
forex|T2
pumpanddump|T2
getrichquick|T2
getrich|T2
quickrich|T2
quickcash|T2
fastcash|T2
fastmoney|T2
surewin|T2
sure-win|T2
riskfree|T2
risk-free|T2
guaranteedreturn|T2
guaranteedprofit|T2
guaranteereturn|T2
investmentclub|T2
signalgroup|T2
botsignal|T2
guarantee|T1
guaranteed|T1
highreturn|T1
highprofit|T1
passiveincome|T1
autoprofit|T1
doublemoney|T1
doubleyourmoney|T1
tripleyourmoney|T1
instantprofit|T1
easyprofit|T1
easycash|T1
win100percent|T1
profit100percent|T1
fakeid|T3
fakeic|T3
borrowedic|T3
borrowedid|T3
stolenid|T3
stolenic|T3
forgedsignature|T3
forgeddocument|T3
counterfeited|T3
stolenaccount|T3
impersonate|T2
impersonation|T2
shellcompany|T2
dummycompany|T2
ghostaccount|T2
frontcompany|T2
nominee|T2
proxy|T2
alias|T2
pseudonym|T2
shell|T1
dummy|T1
terrorist|T3
terrorism|T3
isis|T3
alqaeda|T3
jihad|T3
suicidebomb|T3
martyrdom|T3
extremist|T2
extremism|T2
radical|T2
radicalized|T2
bomber|T2
laundering|T3
launder|T3
washmoney|T3
washcash|T3
offshoreaccount|T3
taxevasion|T3
offshore|T2
taxhaven|T2
haven|T2
unreported|T2
underreported|T2
undeclared|T2
drugbusiness|T3
drugtrade|T3
illicittrade|T3
illegalbusiness|T3
illegalbetting|T3
gamblingwebsite|T3
bettingwebsite|T3
pornsite|T3
adultsite|T3
escortservice|T3
bribemoney|T3
bribefund|T3
corruptionmoney|T3
kickbackmoney|T3
smugglingring|T3
weaponsdeal|T3
gunsdeal|T3
armsdeal|T3
blackmail|T3
extort|T3
extortion|T3
threaten|T2
threat|T2
sabotage|T2
revenge|T2
retaliation|T2
harass|T2
harassment|T2
lawsuit|T2
sue|T2
expose|T1
complainpublic|T1
flamethebank|T1
"""

RISK_DICT = {}
for line in RAW_RISK_DATA.strip().split('\\n'):
    if "|" in line and not line.startswith("#"):
        word, tier = line.split("|")
        RISK_DICT[word.strip().lower()] = tier.strip()

# --- AI MODEL SETUP ---

@st.cache_resource
def load_nlp_model():
    try:
        from transformers import pipeline
        return pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")
    except: return None

classifier = load_nlp_model()

def compute_text_score(text):
    if not text or len(str(text)) < 5: return 50.0, []
    text_lower = str(text).lower()
    detected_flags = []
    max_penalty = 50.0

    for keyword, tier in RISK_DICT.items():
        if keyword in text_lower:
            detected_flags.append(f"{keyword} ({tier})")
            if tier == 'T3': max_penalty = 100.0
            elif tier == 'T2': max_penalty = max(max_penalty, 85.0)
            elif tier == 'T1': max_penalty = max(max_penalty, 65.0)

    if max_penalty == 100.0:
        return 100.0, detected_flags

    if not detected_flags and classifier:
        risk_categories = {
            "investment or education": 20, "home improvement": 30, "business expansion": 25,
            "debt consolidation": 50, "medical emergency": 65,
            "luxury purchase or vacation": 85, "financial distress": 95
        }
        try:
            labels = list(risk_categories.keys())
            result = classifier(str(text), labels, multi_label=False)
            ai_score = 0
            scores = dict(zip(result['labels'], result['scores']))
            for label, prob in scores.items():
                ai_score += prob * risk_categories[label]
            return ai_score, []
        except: return 50.0, []

    return max_penalty, detected_flags

def compute_fusion_risk(numeric_score, text, text_multiplier=0.2):
    text_score, flags = compute_text_score(text)
    risk_factor = (text_score - 50.0) / 50.0
    multiplier = 1.0 + (risk_factor * text_multiplier)
    return multiplier, text_score, flags

def render_risk_gauge(score, title="Total Risk Score"):
    fig = go.Figure(go.Indicator(
        mode = "gauge+number", value = score,
        title = {'text': title, 'font': {'size': 24}},
        domain = {'x': [0, 1], 'y': [0, 1]},
        gauge = {
            'axis': {'range': [0, 100], 'tickwidth': 1, 'tickcolor': "white"},
            'bar': {'color': "rgba(0,0,0,0)"},
            'bgcolor': "white",
            'borderwidth': 2, 'bordercolor': "gray",
            'steps': [
                {'range': [0, 35], 'color': "#00cc96"},
                {'range': [35, 75], 'color': "#ffa15a"},
                {'range': [75, 100], 'color': "#ef553b"}
            ],
            'threshold': {'line': {'color': "white", 'width': 4}, 'thickness': 0.75, 'value': score}
        }
    ))
    fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)", font = {'color': "white", 'family': "Arial"})
    st.plotly_chart(fig, use_container_width=True)

# --- NAVIGATION HELPER ---
def render_nav_buttons():
    st.markdown("---")
    col1, col2 = st.columns([1, 1])
    with col1:
        if st.session_state.current_page > 1:
            if st.button("⬅️ Previous", use_container_width=True):
                st.session_state.current_page -= 1
                st.rerun()
    with col2:
        if st.session_state.current_page < 4:
            if st.button("Next ➡️", use_container_width=True):
                st.session_state.current_page += 1
                st.rerun()

# --- SCORING LOGIC ---
def compute_personal_numeric_risk_scores(data):
    score = 0
    breakdown = {}

    # Calculate Total Income (Weigh Variable Income at 70%)
    weighted_income = data['app_fixed_income'] + (data['app_variable_income'] * 0.7)
    total_commitments = data['app_total_monthly_loan_repayment'] + data['app_other_fixed_monthly_commitments']
    dti = total_commitments / (weighted_income + 1e-6)

    dti_score = 0
    if dti > 0.60: dti_score = 40
    elif dti > 0.45: dti_score = 25
    elif dti > 0.35: dti_score = 15
    score += dti_score
    breakdown['DTI Risk'] = dti_score

    # Housing Risk
    housing_score = 0
    if "Rented" in data['app_housing_status']: housing_score = 10
    elif "Parents" in data['app_housing_status']: housing_score = 5
    score += housing_score
    breakdown['Housing Stability'] = housing_score

    util = data['app_credit_card_outstanding'] / (data['app_total_credit_limit'] + 1e-6)
    util_score = 0
    if util > 0.75: util_score = 25
    elif util > 0.50: util_score = 15
    elif util > 0.30: util_score = 10
    score += util_score
    breakdown['Credit Util Risk'] = util_score

    emp_score = 0
    if data['app_employment_status'] == "Unemployed": emp_score += 30
    elif "SME" in data['app_employment_status']: emp_score += 10
    if data['app_years_in_job'] < 1: emp_score += 10
    score += emp_score
    breakdown['Employment Stability'] = emp_score

    sec_score = 0
    if not data['loan_is_secured']: sec_score = 25
    else:
        ltv = data['loan_amount_requested'] / (data['loan_collateral_value'] + 1e-6)
        if ltv > 0.9: sec_score = 10
    score += sec_score
    breakdown['Collateral Risk'] = sec_score

    if data['app_in_legal_proceedings']: score += 40; breakdown['Legal Penalty'] = 40
    if data['app_convicted_financial_crime']: score += 100; breakdown['Crime Penalty'] = 100

    if data['guar_exists']:
        guar_income_ratio = data['guar_monthly_income'] / (weighted_income + 1e-6)
        if guar_income_ratio > 0.5: score -= 15; breakdown['Guarantor Bonus'] = -15
        elif guar_income_ratio > 0.2: score -= 5; breakdown['Guarantor Bonus'] = -5

    return np.clip(score, 0, 100), breakdown

def compute_business_numeric_score(data):
    score = 0
    breakdown = {}
    explanations = []

    rev_risk = 0
    annual_rev = max(data['biz_rev_year_2'], data['biz_avg_monthly_revenue'] * 12)
    if annual_rev < 100000: rev_risk += 20; explanations.append("⚠️ Micro-Business Revenue (<$100k) +20 Risk")
    elif annual_rev < 300000: rev_risk += 10; explanations.append("Small Scale Revenue +10 Risk")

    # Growth Calculation (Year 2 vs Year 1)
    if data['biz_rev_year_1'] > 0:
        growth = (data['biz_rev_year_2'] - data['biz_rev_year_1']) / data['biz_rev_year_1']
        if growth < -0.15: rev_risk += 20; explanations.append(f"⚠️ Sharp Revenue Decline ({int(growth*100)}%) +20 Risk")
        elif growth < 0: rev_risk += 10; explanations.append("Declining Revenue +10 Risk")

    score += max(0, rev_risk)
    breakdown['Revenue Risk'] = max(0, rev_risk)

    prof_risk = 0
    if data['biz_rev_year_2'] > 0:
        actual_margin = data['biz_profit_year_2'] / data['biz_rev_year_2']
        if actual_margin < 0: prof_risk += 30; explanations.append("⚠️ Business is Loss Making +30 Risk")
        elif actual_margin < 0.03: prof_risk += 15; explanations.append("Very Thin Margins (<3%) +15 Risk")
        elif actual_margin > 0.15: prof_risk -= 10; explanations.append("✅ Healthy Profit Margins -10 Risk")
    score += prof_risk
    breakdown['Profitability Risk'] = prof_risk

    # Liquidity & Banking Check
    liq_risk = 0
    # Implied Turnover Check: Does Bank Stmt match Revenue?
    implied_annual_turnover = data['biz_avg_bank_balance'] * 12 # Rough proxy for turnover velocity
    if implied_annual_turnover < (annual_rev * 0.1): # Very low balance relative to revenue
        liq_risk += 15; explanations.append("⚠️ Low Average Bank Balance vs Revenue +15 Risk")

    monthly_free_cash = data['biz_avg_monthly_revenue'] - data['biz_avg_monthly_expenses'] - data['biz_existing_loan_monthly_repayment']
    est_new_repayment = data['loan_amount_requested'] / (data['loan_tenure_months'] + 1e-6) * 1.1
    if est_new_repayment > 0:
        dscr = monthly_free_cash / est_new_repayment
        if dscr < 1.0: liq_risk += 30; explanations.append(f"⚠️ Insufficient Cashflow (DSCR {dscr:.1f}x) +30 Risk")
        elif dscr < 1.25: liq_risk += 15; explanations.append(f"Tight Cashflow (DSCR {dscr:.1f}x) +15 Risk")

    score += max(0, liq_risk)
    breakdown['Liquidity Risk'] = max(0, liq_risk)

    stab_risk = 0
    if data['biz_credit_score'] < 600: stab_risk += 25; explanations.append("⚠️ Poor Credit Score (<600) +25 Risk")
    elif data['biz_credit_score'] > 750: stab_risk -= 10; explanations.append("✅ Strong Credit Score -10 Risk")

    if data['biz_years_in_operation'] < 3: stab_risk += 10; explanations.append("Young Business (<3 Years) +10 Risk")

    if data['loan_is_secured']:
        ltv = data['loan_amount_requested'] / (data['loan_collateral_value'] + 1e-6)
        if ltv < 0.70: stab_risk -= 20; explanations.append("✅ Fully Secured (LTV <70%) -20 Risk")
    else: stab_risk += 10; explanations.append("Unsecured Loan +10 Risk")
    score += stab_risk
    breakdown['Stability Risk'] = stab_risk

    penalty = 0
    if data['biz_tax_arrears'] or data['biz_epf_arrears']: penalty += 40; explanations.append("❌ Statutory Arrears Detected +40 Risk")
    if data['biz_legal_dispute']: penalty += 30; explanations.append("❌ Active Legal Dispute +30 Risk")
    if data['biz_blacklist_history']: penalty += 100; explanations.append("❌ BLACKLISTED +100 Risk")
    score += penalty
    breakdown['Penalties'] = penalty

    return np.clip(score, 0, 100), breakdown, explanations

def init_state():
    if 'loan_type' not in st.session_state: st.session_state.loan_type = None
    if 'current_page' not in st.session_state: st.session_state.current_page = 1

    defaults = {
        'app_age': 30, 'app_gender': 'Male', 'app_marital_status': 'Single', 'app_housing_status': 'Rented',
        'app_dependents': 0, 'app_education': "Bachelor's Degree", 'app_employment_status': 'Salaried (MNC/Gov)',
        'app_industry': 'Manufacturing', 'app_years_in_job': 2, 'app_total_work_experience': 5,
        'app_fixed_income': 4000, 'app_variable_income': 0, # Split income
        'app_has_mortgage': False, 'app_has_car_loan': False, 'app_has_personal_loan': False,
        'app_has_credit_card': True, 'app_num_credit_cards': 1,
        'app_total_credit_limit': 10000, 'app_credit_card_outstanding': 1000,
        'app_total_monthly_loan_repayment': 500, 'app_other_fixed_monthly_commitments': 200,
        'app_in_legal_proceedings': False, 'app_convicted_financial_crime': False,
        'guar_exists': False, 'guar_relationship': 'Spouse', 'guar_age': 30,
        'guar_employment_status': 'Salaried (SME)', 'guar_monthly_income': 0, 'guar_other_monthly_income': 0,
        'guar_total_monthly_loan_repayment': 0,
        'guar_has_credit_card': False, 'guar_num_credit_cards': 0,
        'guar_total_credit_limit': 0, 'guar_credit_card_outstanding': 0,
        'loan_amount_requested': 20000, 'loan_tenure_months': 24, 'loan_purpose': 'Personal',
        'loan_is_secured': False, 'loan_collateral_type': 'None', 'loan_collateral_value': 0,
        'loan_application_date': date.today(), 'loan_essay_text': "I am applying for this loan to...",

        # Business Defaults
        'biz_name': "", 'biz_reg_no': "", 'biz_entity_type': "Private Ltd (Sdn Bhd)",
        'biz_year_established': 2018, 'biz_industry': 'Retail/F&B',
        'biz_years_in_operation': 5, 'biz_num_owners': 2, 'biz_main_product': "",
        'biz_customer_concentration_high': False, 'biz_num_employees': 10, 'biz_premise_ownership': "Rented",
        'biz_rev_year_1': 500000, 'biz_profit_year_1': 50000, # Previous Year
        'biz_rev_year_2': 600000, 'biz_profit_year_2': 70000, # Current Year
        'biz_avg_monthly_revenue': 50000, 'biz_avg_monthly_expenses': 40000, 'biz_avg_bank_balance': 20000,
        'biz_existing_loan_count': 1, 'biz_existing_loan_outstanding': 50000, 'biz_existing_loan_monthly_repayment': 2000,
        'biz_has_overdraft': False, 'biz_overdraft_limit': 0, 'biz_overdraft_utilization': 0,
        'biz_tax_arrears': False, 'biz_epf_arrears': False, 'biz_credit_score': 750,
        'has_personal_guarantee': False, 'guar_owner_income': 0, 'guar_owner_existing_debt': 0,
        'biz_legal_dispute': False, 'biz_blacklist_history': False, 'biz_high_risk_sector': False,
        'biz_essay_text': "The purpose of this facility is to...",

        # Results
        'numeric_score': 0, 'text_score': 0, 'risk_score': 0, 'score_multiplier': 1.0,
        'risk_category': 'N/A', 'risk_breakdown': {}, 'risk_explanations': [], 'detected_flags': []
    }
    if 'form_data' not in st.session_state: st.session_state.form_data = defaults
    else:
        for key, value in defaults.items():
            if key not in st.session_state.form_data: st.session_state.form_data[key] = value

# --- PAGE RENDERERS ---

def page_loan_type_selection():
    st.header("Start Application")
    st.info("Please select the type of loan you are applying for.")
    col1, col2 = st.columns(2)
    if col1.button("👤 Personal Loan", use_container_width=True):
        st.session_state.loan_type = "Personal"
        st.rerun()
    if col2.button("🏢 Business Loan", use_container_width=True):
        st.session_state.loan_type = "Business"
        st.rerun()

def personal_page_1():
    st.header("1. Applicant Profile & Income")

    st.subheader("Identity & Residence")
    c1, c2, c3 = st.columns(3)
    st.session_state.form_data['app_age'] = c1.number_input("Age", 18, 100, st.session_state.form_data['app_age'])
    st.session_state.form_data['app_gender'] = c2.selectbox("Gender", GENDER_OPTIONS, index=0)
    st.session_state.form_data['app_marital_status'] = c3.selectbox("Marital Status", MARITAL_OPTIONS, index=0)

    c4, c5 = st.columns(2)
    st.session_state.form_data['app_education'] = c4.selectbox("Highest Education", EDUCATION_OPTIONS, index=2)
    st.session_state.form_data['app_housing_status'] = c5.selectbox("Residential Status", HOUSING_OPTIONS, index=2)

    st.markdown("---")
    st.subheader("Employment Details")
    e1, e2 = st.columns(2)
    st.session_state.form_data['app_employment_status'] = e1.selectbox("Employment Type", EMPLOYMENT_OPTIONS, index=0)
    st.session_state.form_data['app_industry'] = e2.selectbox("Industry/Sector", list(INDUSTRY_RISK.keys()), index=4)

    e3, e4 = st.columns(2)
    st.session_state.form_data['app_years_in_job'] = e3.number_input("Years in Current Job", 0.0, 50.0, st.session_state.form_data['app_years_in_job'] * 1.0, step=0.5)
    st.session_state.form_data['app_total_work_experience'] = e4.number_input("Total Working Experience (Years)", 0.0, 50.0, st.session_state.form_data['app_total_work_experience'] * 1.0, step=0.5)

    st.markdown("---")
    st.subheader("Monthly Income (Net)")
    i1, i2 = st.columns(2)
    st.session_state.form_data['app_fixed_income'] = i1.number_input("Fixed Basic Salary ($)", 0, value=st.session_state.form_data['app_fixed_income'], step=100, help="Base salary excluding allowances/OT")
    st.session_state.form_data['app_variable_income'] = i2.number_input("Variable Income (Avg Commission/OT)", 0, value=st.session_state.form_data['app_variable_income'], step=100, help="Average of last 6 months")

    st.subheader("Monthly Commitments")
    k1, k2 = st.columns(2)
    st.session_state.form_data['app_total_monthly_loan_repayment'] = k1.number_input("Existing Bank Loan Repayments ($)", 0, value=st.session_state.form_data['app_total_monthly_loan_repayment'], step=50, help="Total for Car, Housing, Personal Loans")
    st.session_state.form_data['app_other_fixed_monthly_commitments'] = k2.number_input("Other Fixed Commitments ($)", 0, value=st.session_state.form_data['app_other_fixed_monthly_commitments'], step=50, help="Insurance, PTPTN, Co-op loans")

    render_nav_buttons()

def personal_page_2():
    st.header("2. Credit Profile & Guarantor")

    st.subheader("Credit Card Usage")
    c1, c2, c3 = st.columns(3)
    st.session_state.form_data['app_num_credit_cards'] = c1.number_input("Number of Credit Cards", 0, 20, value=st.session_state.form_data['app_num_credit_cards'])
    st.session_state.form_data['app_total_credit_limit'] = c2.number_input("Combined Credit Limit ($)", 0, value=st.session_state.form_data['app_total_credit_limit'], step=1000)
    st.session_state.form_data['app_credit_card_outstanding'] = c3.number_input("Current Outstanding Balance ($)", 0, value=st.session_state.form_data['app_credit_card_outstanding'], step=100)

    st.markdown("---")
    st.subheader("Legal & Statutory")
    l1, l2 = st.columns(2)
    st.session_state.form_data['app_in_legal_proceedings'] = l1.checkbox("Are you currently under legal proceedings (Bankruptcy/Litigation)?", st.session_state.form_data['app_in_legal_proceedings'])
    st.session_state.form_data['app_convicted_financial_crime'] = l2.checkbox("Have you ever been convicted of a financial crime?", st.session_state.form_data['app_convicted_financial_crime'])

    st.markdown("---")
    st.subheader("Guarantor Details")
    st.session_state.form_data['guar_exists'] = st.radio("Is a Guarantor available for this facility?", [True, False], index=0 if st.session_state.form_data['guar_exists'] else 1)

    if st.session_state.form_data['guar_exists']:
        g1, g2, g3 = st.columns(3)
        st.session_state.form_data['guar_relationship'] = g1.selectbox("Relationship to Applicant", RELATIONSHIP_OPTIONS, index=0)
        st.session_state.form_data['guar_age'] = g2.number_input("Guarantor Age", 21, 80, st.session_state.form_data['guar_age'])
        st.session_state.form_data['guar_employment_status'] = g3.selectbox("Guarantor Employment", EMPLOYMENT_OPTIONS, index=1)

        g4, g5 = st.columns(2)
        st.session_state.form_data['guar_monthly_income'] = g4.number_input("Guarantor Net Monthly Income ($)", 0, value=st.session_state.form_data['guar_monthly_income'], step=500)
        st.session_state.form_data['guar_total_monthly_loan_repayment'] = g5.number_input("Guarantor Monthly Commitments ($)", 0, value=st.session_state.form_data['guar_total_monthly_loan_repayment'], step=100)

    render_nav_buttons()

def personal_page_3():
    st.header("3. Loan Request & Collateral")

    l1, l2 = st.columns(2)
    st.session_state.form_data['loan_amount_requested'] = l1.number_input("Loan Amount Requested ($)", 1000, value=st.session_state.form_data['loan_amount_requested'], step=1000)
    st.session_state.form_data['loan_tenure_months'] = l2.number_input("Repayment Tenure (Months)", 6, 84, value=st.session_state.form_data['loan_tenure_months'])

    st.subheader("Security/Collateral")
    st.session_state.form_data['loan_is_secured'] = st.toggle("I am providing collateral for this loan", value=st.session_state.form_data['loan_is_secured'])

    if st.session_state.form_data['loan_is_secured']:
        c1, c2 = st.columns(2)
        st.session_state.form_data['loan_collateral_type'] = c1.selectbox("Collateral Type", COLLATERAL_OPTIONS, index=0)
        st.session_state.form_data['loan_collateral_value'] = c2.number_input("Estimated Market Value ($)", 0, value=st.session_state.form_data['loan_collateral_value'], step=5000)

    st.markdown("---")
    st.subheader("Purpose Statement")
    st.info("Please explain exactly how you intend to use these funds. Be specific.")
    st.session_state.form_data['loan_essay_text'] = st.text_area("Purpose:", value=st.session_state.form_data['loan_essay_text'], height=150, placeholder="e.g., Debt consolidation of 3 credit cards and renovation of master bedroom...")

    render_nav_buttons()

def personal_page_4(mult):
    st.header("Risk Analysis Report")
    if st.button("Generate Assessment", type="primary"):
        with st.spinner("Processing credit logic and NLP models..."):
            numeric_score, breakdown = compute_personal_numeric_risk_scores(st.session_state.form_data)
            st.session_state.form_data['numeric_score'] = numeric_score
            st.session_state.form_data['risk_breakdown'] = breakdown

            multiplier, text_score, flags = compute_fusion_risk(numeric_score, st.session_state.form_data['loan_essay_text'], mult)
            final_risk = numeric_score * multiplier

            if text_score >= 99: final_risk = 98.0; st.session_state.form_data['score_multiplier'] = 99.0

            st.session_state.form_data['risk_score'] = np.clip(final_risk, 0, 100)
            st.session_state.form_data['score_multiplier'] = multiplier if text_score < 99 else 99.0
            st.session_state.form_data['text_score'] = text_score
            st.session_state.form_data['detected_flags'] = flags

            if final_risk >= 75: st.session_state.form_data['risk_category'] = "HIGH RISK (DECLINE)"
            elif final_risk >= 35: st.session_state.form_data['risk_category'] = "MEDIUM RISK (KIV/REVIEW)"
            else: st.session_state.form_data['risk_category'] = "LOW RISK (APPROVE)"
            time.sleep(1)
        st.success("Assessment Complete")

        status = st.session_state.form_data['risk_category']
        color = "red" if "HIGH" in status else "orange" if "MEDIUM" in status else "green"
        st.markdown(f"<h1 style='text-align: center; color: {color};'>{status}</h1>", unsafe_allow_html=True)
        st.markdown("---")

        col_gauge, col_details = st.columns([1, 1])
        with col_gauge: render_risk_gauge(st.session_state.form_data['risk_score'])
        with col_details:
            st.subheader("Risk Drivers")
            flags = st.session_state.form_data['detected_flags']
            if flags:
                st.error("🚩 **High Risk Keywords:**")
                for f in flags: st.write(f"- {f}")
            else: st.success("✅ No adverse keywords detected.")
            st.markdown("### Score Composition")
            st.write(f"**Financial Score:** {st.session_state.form_data['numeric_score']:.1f}")
            st.write(f"**NLP Sentiment Score:** {st.session_state.form_data['text_score']:.1f}")
            st.write(f"**Risk Multiplier:** x{st.session_state.form_data['score_multiplier']:.2f}")

        st.markdown("### Component Breakdown")
        df = pd.DataFrame([{"Component": k, "Points": v} for k, v in st.session_state.form_data['risk_breakdown'].items()])
        st.dataframe(df, use_container_width=True)
    render_nav_buttons()

def business_page_1():
    st.header("1. Business Profile")

    st.subheader("Entity Details")
    c1, c2 = st.columns(2)
    st.session_state.form_data['biz_name'] = c1.text_input("Registered Business Name", value=st.session_state.form_data['biz_name'])
    st.session_state.form_data['biz_reg_no'] = c2.text_input("Registration No (SSM)", value=st.session_state.form_data['biz_reg_no'])

    c3, c4 = st.columns(2)
    st.session_state.form_data['biz_entity_type'] = c3.selectbox("Legal Constitution", ["Sole Proprietorship", "Partnership", "Private Ltd (Sdn Bhd)", "Public Ltd (Bhd)"], index=2)
    st.session_state.form_data['biz_year_established'] = c4.number_input("Year of Incorporation", 1950, 2025, value=st.session_state.form_data['biz_year_established'])

    st.subheader("Operations")
    c5, c6 = st.columns(2)
    st.session_state.form_data['biz_industry'] = c5.selectbox("Industry Sector", list(INDUSTRY_RISK.keys()), index=5)
    st.session_state.form_data['biz_years_in_operation'] = c6.number_input("Years in Active Operation", 0, 100, value=st.session_state.form_data['biz_years_in_operation'])

    c7, c8 = st.columns(2)
    st.session_state.form_data['biz_num_employees'] = c7.number_input("Total Number of Employees", 1, 10000, value=st.session_state.form_data['biz_num_employees'])
    st.session_state.form_data['biz_premise_ownership'] = c8.selectbox("Business Premise Status", ["Rented", "Owned (Encumbered)", "Owned (Free)", "Virtual/Shared Office"], index=0)

    render_nav_buttons()

def business_page_2():
    st.header("2. Financial Performance")

    st.subheader("Audited Financials (Profit & Loss)")
    st.caption("Please enter figures from your most recent audited accounts or management accounts.")

    col_y1, col_y2 = st.columns(2)
    with col_y1:
        st.markdown("**Previous Year (Y-1)**")
        st.session_state.form_data['biz_rev_year_1'] = st.number_input("Revenue Y-1 ($)", 0, value=st.session_state.form_data['biz_rev_year_1'], step=10000)
        st.session_state.form_data['biz_profit_year_1'] = st.number_input("Net Profit/Loss Y-1 ($)", value=st.session_state.form_data['biz_profit_year_1'], step=5000)

    with col_y2:
        st.markdown("**Most Recent Year (Y)**")
        st.session_state.form_data['biz_rev_year_2'] = st.number_input("Revenue Y ($)", 0, value=st.session_state.form_data['biz_rev_year_2'], step=10000)
        st.session_state.form_data['biz_profit_year_2'] = st.number_input("Net Profit/Loss Y ($)", value=st.session_state.form_data['biz_profit_year_2'], step=5000)

    st.markdown("---")
    st.subheader("Current Cashflow & Banking")
    c1, c2, c3 = st.columns(3)
    st.session_state.form_data['biz_avg_monthly_revenue'] = c1.number_input("Avg Monthly Sales (Last 6m)", 0, value=st.session_state.form_data['biz_avg_monthly_revenue'], step=1000)
    st.session_state.form_data['biz_avg_monthly_expenses'] = c2.number_input("Avg Monthly Opex (Last 6m)", 0, value=st.session_state.form_data['biz_avg_monthly_expenses'], step=1000)
    st.session_state.form_data['biz_avg_bank_balance'] = c3.number_input("Avg End Month Bank Balance", 0, value=st.session_state.form_data['biz_avg_bank_balance'], step=1000)

    st.subheader("Liabilities")
    d1, d2, d3 = st.columns(3)
    st.session_state.form_data['biz_existing_loan_count'] = d1.number_input("Count of Active Loans", 0, value=st.session_state.form_data['biz_existing_loan_count'])
    st.session_state.form_data['biz_existing_loan_outstanding'] = d2.number_input("Total Principal Outstanding ($)", 0, value=st.session_state.form_data['biz_existing_loan_outstanding'], step=5000)
    st.session_state.form_data['biz_existing_loan_monthly_repayment'] = d3.number_input("Total Monthly Debt Service ($)", 0, value=st.session_state.form_data['biz_existing_loan_monthly_repayment'], step=1000)

    render_nav_buttons()

def business_page_3():
    st.header("3. Facility Request")

    st.subheader("Loan Terms")
    # REMOVED PURPOSE DROPDOWN
    l1, l2 = st.columns(2)
    st.session_state.form_data['loan_amount_requested'] = l1.number_input("Facility Amount Required ($)", 10000, value=st.session_state.form_data['loan_amount_requested'], step=10000)
    st.session_state.form_data['loan_tenure_months'] = l2.number_input("Tenure (Months)", 6, 120, value=st.session_state.form_data['loan_tenure_months'])

    st.subheader("Collateral & Risk Mitigation")
    st.session_state.form_data['loan_is_secured'] = st.checkbox("Facility will be secured by asset?", value=st.session_state.form_data['loan_is_secured'])
    if st.session_state.form_data['loan_is_secured']:
        s1, s2 = st.columns(2)
        st.session_state.form_data['loan_collateral_type'] = s1.selectbox("Asset Type", ["Industrial Land", "Commercial Building", "Fixed Deposit", "Machinery/Equipment"], index=0)
        st.session_state.form_data['loan_collateral_value'] = s2.number_input("Open Market Value ($)", 0, value=st.session_state.form_data['loan_collateral_value'], step=10000)

    st.markdown("---")
    st.subheader("Director/Shareholder Declarations")
    c_score_col, _ = st.columns(2)
    st.session_state.form_data['biz_credit_score'] = c_score_col.number_input("Director's CTOS/Credit Score", 300, 850, value=st.session_state.form_data['biz_credit_score'])

    c_leg, c_black, c_stat = st.columns(3)
    st.session_state.form_data['biz_legal_dispute'] = c_leg.checkbox("Active Legal Suits?", value=st.session_state.form_data['biz_legal_dispute'])
    st.session_state.form_data['biz_blacklist_history'] = c_black.checkbox("Blacklist Record?", value=st.session_state.form_data['biz_blacklist_history'])
    st.session_state.form_data['biz_tax_arrears'] = c_stat.checkbox("Tax/EPF Arrears?", value=st.session_state.form_data['biz_tax_arrears'])

    st.subheader("Utilization Plan (Essay)")
    st.caption("Describe your business plan and utilization of funds. This text is analyzed for risk intent.")
    st.session_state.form_data['biz_essay_text'] = st.text_area("Details:", value=st.session_state.form_data['biz_essay_text'], height=150)
    render_nav_buttons()

def business_page_results(mult):
    st.header("Business Risk Analysis")
    if st.button("Calculate Business Risk", type="primary"):
        with st.spinner("Analyzing Financial Ratios & Sentiment..."):
            numeric_score, breakdown, explanations = compute_business_numeric_score(st.session_state.form_data)
            st.session_state.form_data['numeric_score'] = numeric_score
            st.session_state.form_data['risk_breakdown'] = breakdown
            st.session_state.form_data['risk_explanations'] = explanations

            multiplier, text_score, flags = compute_fusion_risk(numeric_score, st.session_state.form_data['biz_essay_text'], mult)
            final_risk = numeric_score * multiplier

            if text_score >= 99: final_risk = 98.0; st.session_state.form_data['score_multiplier'] = 99.0

            st.session_state.form_data['risk_score'] = np.clip(final_risk, 0, 100)
            st.session_state.form_data['score_multiplier'] = multiplier if text_score < 99 else 99.0
            st.session_state.form_data['text_score'] = text_score
            st.session_state.form_data['detected_flags'] = flags

            if final_risk >= 75: st.session_state.form_data['risk_category'] = "HIGH RISK (DECLINE)"
            elif final_risk >= 35: st.session_state.form_data['risk_category'] = "MEDIUM RISK (REVIEW)"
            else: st.session_state.form_data['risk_category'] = "LOW RISK (APPROVE)"
            time.sleep(1)

        st.success("Assessment Complete")

        status = st.session_state.form_data['risk_category']
        color = "red" if "HIGH" in status else "orange" if "MEDIUM" in status else "green"
        st.markdown(f"<h1 style='text-align: center; color: {color};'>{status}</h1>", unsafe_allow_html=True)
        st.markdown("---")

        col_gauge, col_details = st.columns([1, 1])
        with col_gauge: render_risk_gauge(st.session_state.form_data['risk_score'])

        with col_details:
            st.subheader("Risk Factors")
            flags = st.session_state.form_data['detected_flags']
            if flags:
                st.error("🚩 **Critical Keywords:**")
                for f in flags: st.write(f"- {f}")
            else: st.success("✅ No critical keywords found.")
            st.markdown("### Score Composition")
            st.write(f"**Financial Score:** {st.session_state.form_data['numeric_score']:.1f}")
            st.write(f"**NLP Score:** {st.session_state.form_data['text_score']:.1f}")
            st.write(f"**Multiplier:** x{st.session_state.form_data['score_multiplier']:.2f}")

        st.markdown("### Detailed Analysis")
        df = pd.DataFrame([{"Component": k, "Risk Points": f"{v:.1f}"} for k, v in st.session_state.form_data['risk_breakdown'].items()])
        st.dataframe(df, use_container_width=True)
        with st.expander("🔎 View Specific Risk Explanations"):
            if st.session_state.form_data.get('risk_explanations'):
                for exp in st.session_state.form_data['risk_explanations']:
                    if "+" in exp: st.markdown(f":red[**{exp}**]")
                    elif "-" in exp: st.markdown(f":green[**{exp}**]")
                    else: st.write(exp)
            else: st.write("No specific risk drivers identified.")
    render_nav_buttons()

def main():
    init_state()
    st.set_page_config(page_title="Unified Loan App", layout="wide")

    with st.sidebar:
        st.title("Navigation")
        if st.button("🏠 Home / Reset"):
            st.session_state.loan_type = None
            st.session_state.current_page = 1
            st.rerun()

        st.markdown("---")
        st.header("⚙️ Risk Control")
        st.info("Adjust text analysis sensitivity.")
        mode = st.radio("Sensitivity", ["Standard (0.2x)", "Aggressive (0.5x)", "Custom"], index=0)
        if mode == "Standard (0.2x)": mult = 0.2
        elif mode == "Aggressive (0.5x)": mult = 0.5
        else: mult = st.slider("Multiplier", 0.0, 1.0, 0.2, 0.05)

    if st.session_state.loan_type is None:
        page_loan_type_selection()
    elif st.session_state.loan_type == "Personal":
        if st.session_state.current_page == 1: personal_page_1()
        elif st.session_state.current_page == 2: personal_page_2()
        elif st.session_state.current_page == 3: personal_page_3()
        elif st.session_state.current_page == 4: personal_page_4(mult)
    elif st.session_state.loan_type == "Business":
        if st.session_state.current_page == 1: business_page_1()
        elif st.session_state.current_page == 2: business_page_2()
        elif st.session_state.current_page == 3: business_page_3()
        elif st.session_state.current_page == 4: business_page_results(mult)

if __name__ == "__main__": main()
'''

with open('app.py', 'w') as f: f.write(loan_risk_app_code)
print("✅ Application file saved.")

# ==========================================
# PART 3: LAUNCH SYSTEM
# ==========================================

PORT = 8501

print(f"\\n🚀 Launching Streamlit on port {PORT}...")
st_log = open("streamlit.log", "w")
app_process = subprocess.Popen(
    f"streamlit run app.py --server.port {PORT} --server.address 0.0.0.0 --server.headless true",
    shell=True,
    preexec_fn=os.setsid,
    stdout=st_log,
    stderr=st_log
)

print("⏳ Waiting for app to initialize (this prevents 'Site Cant Be Reached')...")
print("   (This might take 1 minute to download AI models)")
ready = False
for i in range(60):
    try:
        with urllib.request.urlopen(f"http://localhost:{PORT}/_stcore/health", timeout=1) as response:
            if response.status == 200:
                print("\\n✅ Streamlit is ready!")
                ready = True
                break
    except:
        time.sleep(1)
        if i % 5 == 0: print(".", end="", flush=True)

if not ready:
    print("\\n❌ Error: App failed to start. Checking logs...")
    if os.path.exists("streamlit.log"):
        with open("streamlit.log", "r") as f:
            print(f.read()[-2000:])
    else:
        print("No log file found.")
else:
    if not os.path.exists("cloudflared-linux-amd64.deb"):
        print("⬇️ Downloading Cloudflare Tunnel...")
        subprocess.run("wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb", shell=True)
        subprocess.run("dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1", shell=True)

    print("🚇 Starting Tunnel...")
    log_file = "cf.log"
    log = open(log_file, "w")
    tunnel_proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"],
        stdout=log,
        stderr=log,
        preexec_fn=os.setsid
    )

    print("🔗 Generating link...")
    time.sleep(5)
    found_url = False
    try:
        while True:
            if app_process.poll() is not None:
                print("\\n❌ Streamlit app died! Check streamlit.log")
                break
            if tunnel_proc.poll() is not None:
                print("\\n❌ Cloudflare Tunnel died! Check cf.log")
                break

            if not found_url and os.path.exists(log_file):
                with open(log_file, "r") as f:
                    txt = f.read()
                    match = re.search(r'(https:\/\/[a-zA-Z0-9-]+\.trycloudflare\.com)', txt)
                    if match:
                        url = match.group(1)
                        print(f"\\n🎯 \\033[1;32mYOUR APP IS LIVE:\\033[0m {url}")
                        print("⚠️ Keep this cell running to maintain the connection!")
                        found_url = True
            time.sleep(2)
    except KeyboardInterrupt:
        print("\\n🛑 Stopping processes...")
        tunnel_proc.terminate()
        app_process.terminate()

🧹 Cleaning up old processes...
📦 Installing dependencies...
📝 Writing application file...
✅ Application file saved.
\n🚀 Launching Streamlit on port 8501...
⏳ Waiting for app to initialize (this prevents 'Site Cant Be Reached')...
   (This might take 1 minute to download AI models)
.\n✅ Streamlit is ready!
🚇 Starting Tunnel...
🔗 Generating link...
\n🎯 \033YOUR APP IS LIVE:\033 https://treating-doubt-freebsd-feels.trycloudflare.com
⚠️ Keep this cell running to maintain the connection!
